### **Скрипт-инструкция к Logs API Яндекс.Метрики по выгрузке сырых данных в среду Python**

**Введение**

Настоящая скрипт-инструкция по обращению к Logs API Яндекс.Метрики позволяет получать неагрегированные данные, собранные определенным счетчиком Яндекс.Метрики, и преобразовывать их в формат Pandas DataFrame для работы в среде Python, где мы можем самостоятельно обрабатывать статистические данные и использовать их для решения уникальных аналитических задач.

Справка Яндекс.Метрики о Logs API: https://yandex.ru/dev/metrika/doc/api2/logs/intro.html

Для работы с Logs API Яндекс.Метрики в настоящей скрипт-инструкции используется библиотека **tapi_yandex_metrika**, документацию о которой можно рассмотреть по ссылке: https://github.com/pavelmaksimov/tapi-yandex-metrika

**Блок объявления и загрузки библиотек**

In [ ]:
!pip install tapi_yandex_metrika
import pandas as pd
from tapi_yandex_metrika import YandexMetrikaLogsapi
import requests
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Блок объявления токена API и номера счетчика Яндекс.Метрики**
_____________________________

In [ ]:
ACCESS_TOKEN = 'y0_*******************************************Q4' # Введите токен API
COUNTER_ID = '*******' # Введите номер счетчика Яндекс.Метрики

Видео о том, как получить токен API (API_key) Яндекс Метрики: https://www.youtube.com/watch?v=_xCh8LHEJJA

*ВАЖНО: публикация токена API в открытый доступ запрещена, в противном случае третьи лица могут получить полный доступ к счетчику Яндекс.Метрики, в том числе к его настройкам и содержащимся в нем данным*

**Блок объявления параметров запроса к Logs API Яндекс.Метрики**
_______________________________

 - Задаем поля необходимого нам запроса в случае, если хотим смотреть логи визитов **(source : visits)**

*Визит — последовательность действий (активность) одного посетителя на сайте (на одном счетчике)*

 

In [ ]:
fields_visits = "ym:s:date,ym:s:visitID,ym:s:counterID" # ,ym:s:counterID,ym:s:isNewUser,ym:s:pageViews,ym:s:referer,ym:s:UTMSource и т.д.

Список возможных полей для запроса в случае источника логов visits (визиты) приведен по ссылке: https://yandex.ru/dev/metrika/doc/api2/logs/fields/visits.html
_______________________________

- Задаем поля необходимого нам запроса в случае, если хотим смотреть логи просмотров **(source : hits)**

*Просмотр — загрузка страницы сайта при переходе посетителя на нее*

In [ ]:
fields_hits = "ym:pv:date,ym:pv:title,ym:pv:UTMCampaign,ym:pv:UTMSource" # ,ym:pv:deviceCategory,ym:pv:lastTrafficSource и т.д.

Список возможных полей для запроса в случае источника логов hits (просмотры) приведен по ссылке: https://yandex.ru/dev/metrika/doc/api2/logs/fields/hits.html
____________________________________

*Примечание: Logs API Метрики поддерживает механизм задания параметров полей — параметризацию. Возможность доступна для большинства полей (и fields_visits, и fields_hits). С помощью параметризации можно указать модель атрибуции. Также доступны значения, определяющие кросс-девайс атрибуцию. По умолчанию используется lastsign — последний значимый источник. Подробнее о параметризации полей здесь: https://yandex.ru/dev/metrika/doc/api2/logs/param.html*

Задаем временные границы, за которые необходимо получить данные 

In [ ]:
start_date = '2022-10-01'
end_date = '2022-10-02'

**Блок выгрузки данных**
_________________________________

Инициализация параметров подключения к счетчику Яндекс.Метрики

In [ ]:
client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': COUNTER_ID},
    wait_report=True
)
params={
    "fields": fields_visits, #ИЛИ fields_hits
    "source": "visits", #ИЛИ "hits"
    "date1": start_date,
    "date2": end_date
}

Проверка инициализации параметров запроса

In [ ]:
params

Проверка возможности выгрузки отчета с данными, т.к. существует  ограничение на максимальный размер отчета (до 10 Гб), ждем ответ **possible:True**

In [ ]:
result = client.evaluate().get(params=params)
print(result)

<TapiClient object
{   'log_request_evaluation': {   'expected_size': 2965053,
                                  'log_request_sum_max_size': 10737418240,
                                  'log_request_sum_size': 10726398462,
                                  'max_possible_day_quantity': 7235,
                                  'possible': True}}>


Выгрузка данных из Яндекс.Метрики в Pandas DataFrame

In [ ]:
info = client.create().post(params=params)
request_id = info["log_request"]["request_id"]

report = client.download(requestId=request_id).get()

raw = report().to_dicts()
df = pd.DataFrame.from_dict(raw)

Выводим получившийся Pandas DataFrame

In [ ]:
df

,ym:s:date,ym:s:visitID,ym:s:counterID
0,2022-10-01,5767097095466516512,37655510
1,2022-10-01,5767124245926117703,37655510
2,2022-10-02,5784460103729545319,37655510
3,2022-10-01,5765106810708557951,37655510
4,2022-10-02,5781989152076071189,37655510
...,...,...,...
66362,2022-10-01,5770121536926384420,37655510
66363,2022-10-01,5764944333987119336,37655510
66364,2022-10-01,5761231597532086305,37655510
66365,2022-10-01,5766495183310946668,37655510
